In [1]:
!pip install requests

In [5]:
import numpy as np
import pandas as pd
import requests

In [6]:
url = "https://cea.nic.in/api/electricEnergySales.php"

In [9]:
# request the data from server
response = requests.get(url)

In [12]:
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [13]:
retry_strategy = Retry(
    total=3,
    backoff_factor=0.5,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)

adapter = HTTPAdapter(max_retries=retry_strategy)

http  = requests.Session()

http.mount("https://", adapter)
http.mount("http://", adapter)

try:
    response = http.get(url, timeout=10)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        print(df.head())
    else:
        print(f"Request failed with status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

     ID     Year         State region  domestic commercial  \
0  1003  2012-13       Mizoram    NER    197.59       21.1   
1   984  2012-13  D & N Haveli     WR      71.8      27.79   
2   983  2012-13   Daman & Diu     WR     77.31       38.5   
3   982  2012-13           Goa     WR       835        216   
4   981  2012-13   Maharashtra     WR  22594.63   12885.88   

  industrial_low_volt industrial_high_volt public_lighting traction  \
0                1.81                 1.47            5.36        0   
1              163.31              4323.54            5.04        0   
2              160.22               1527.8            6.46        0   
3              127.03              1765.99              15        0   
4             6489.92              35324.5         1258.55  2341.92   

  agriculture public_water_works miscellaneous total_energy_sold  
0        0.08              37.23         21.96             286.6  
1        3.55                  0          2.05           4597.08  

In [14]:
df.head()

,ID,Year,State,region,domestic,commercial,industrial_low_volt,industrial_high_volt,public_lighting,traction,agriculture,public_water_works,miscellaneous,total_energy_sold
0,1003,2012-13,Mizoram,NER,197.59,21.1,1.81,1.47,5.36,0,0.08,37.23,21.96,286.6
1,984,2012-13,D & N Haveli,WR,71.8,27.79,163.31,4323.54,5.04,0,3.55,0,2.05,4597.08
2,983,2012-13,Daman & Diu,WR,77.31,38.5,160.22,1527.8,6.46,0,4.19,1.11,0,1815.59
3,982,2012-13,Goa,WR,835,216,127.03,1765.99,15,0,21,101,0,3081.02
4,981,2012-13,Maharashtra,WR,22594.63,12885.88,6489.92,35324.5,1258.55,2341.92,22059.05,1784.04,174.67,104913.16


In [16]:
# seeing the number of rows and columns
df.shape

(250, 14)

In [17]:
# checking the null values in data
df.isnull().sum()

ID                      0
Year                    0
State                   0
region                  0
domestic                0
commercial              0
industrial_low_volt     0
industrial_high_volt    0
public_lighting         0
traction                0
agriculture             0
public_water_works      0
miscellaneous           0
total_energy_sold       0
dtype: int64

In [19]:
# checking number of records per state
df.groupby('State').size().reset_index(name='Total Records')

,State,Total Records
0,A & N Islands,6
1,A. & N. Islands,1
2,Andhra Pradesh,7
3,Arunachal Pradesh,7
4,Assam,7
5,Bihar,7
6,Chandigarh,7
7,Chhattisgarh,7
8,D & N Haveli,6
9,D. & N. Haveli,1


In [ ]:
# there are multiple records of same states because of different spellings

In [22]:
df['State'] = df['State'].replace({r'[\.\$]':''}, regex=True)

In [23]:
df.groupby('State').size().reset_index(name='Total Records')

,State,Total Records
0,A & N Islands,7
1,Andhra Pradesh,7
2,Arunachal Pradesh,7
3,Assam,7
4,Bihar,7
5,Chandigarh,7
6,Chhattisgarh,7
7,D & N Haveli,7
8,Daman & Diu,7
9,Delhi,7


In [25]:
# convert state to string then remove any spaces(strip)
# replacing wrong spellings with correct spellings
df['State'] = df['State'].str.strip()
df['State'] =  df['State'].replace('Madhay Pradesh','Madhya Pradesh')
df['State'] =  df['State'].replace('Orissa','Odisha')

In [26]:
df.groupby('State').size().reset_index(name='Total Records')

,State,Total Records
0,A & N Islands,7
1,Andhra Pradesh,7
2,Arunachal Pradesh,7
3,Assam,7
4,Bihar,7
5,Chandigarh,7
6,Chhattisgarh,7
7,D & N Haveli,7
8,Daman & Diu,7
9,Delhi,7


In [27]:
df.head()

,ID,Year,State,region,domestic,commercial,industrial_low_volt,industrial_high_volt,public_lighting,traction,agriculture,public_water_works,miscellaneous,total_energy_sold
0,1003,2012-13,Mizoram,NER,197.59,21.1,1.81,1.47,5.36,0,0.08,37.23,21.96,286.6
1,984,2012-13,D & N Haveli,WR,71.8,27.79,163.31,4323.54,5.04,0,3.55,0,2.05,4597.08
2,983,2012-13,Daman & Diu,WR,77.31,38.5,160.22,1527.8,6.46,0,4.19,1.11,0,1815.59
3,982,2012-13,Goa,WR,835,216,127.03,1765.99,15,0,21,101,0,3081.02
4,981,2012-13,Maharashtra,WR,22594.63,12885.88,6489.92,35324.5,1258.55,2341.92,22059.05,1784.04,174.67,104913.16


In [28]:
df.tail()

,ID,Year,State,region,domestic,commercial,industrial_low_volt,industrial_high_volt,public_lighting,traction,agriculture,public_water_works,miscellaneous,total_energy_sold
245,776,2020-21,Lakshadweep,SR,40.347579,11.471602,0.371144,0,0.817116,0,0,0,0.094223,53.101664
246,775,2020-21,Puducherry,SR,842.42999999999,189.33,164.94,1432.4112590958,23.23,0,68.569999999999,0,14.5,2735.4112590958
247,774,2020-21,Tamil Nadu,SR,32721.9,8976.93439072,9332.12571665,31073.292036425,1291.89,564.5,13974.676133,1291.89,2926.06457517,102153.27285196
248,773,2020-21,Kerala,SR,12765.35023315,4267.6231652700,1144.9873619859,3841.1333280863,379.74669930000,168.694773,412.0184,0.251,534.29254524000,23514.097506032
249,754,2020-21,Chandigarh,NR,695.0048,334.309,120.3378,98.393862957765,12.14,0,1.366,53.785,38.266000000000,1353.6024629577


In [29]:
df.to_csv('energy_sales.csv')